In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adadelta
from keras import backend as K
from tensorflow.keras.optimizers import RMSprop
import numpy as np

In [15]:
#permutation experiments
arr1 = np.array([[1, 2,3],[4, 5,6], [7, 8,9]])
arr_flatten = arr1.reshape(9,1)
shuffler = np.random.permutation(len(arr_flatten))
arr1_shuffle = arr1.reshape(9,1)[shuffler].reshape((3, 3))
print(arr1_shuffle)

[[1 4 7]
 [3 5 2]
 [8 6 9]]


In [ ]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
shuffler = np.random.permutation(28*28)
for i, s in enumerate(x_train):
  x_train[i] = x_train[i].reshape(28*28,1)[shuffler].reshape((28, 28))
for i, s in enumerate(x_test):
  x_test[i] = x_test[i].reshape(28*28,1)[shuffler].reshape((28, 28))

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
469/469 [==============================] - 8s 16ms/step - loss: 2.2850 - accuracy: 0.1276 - val_loss: 2.2635 - val_accuracy: 0.2013
Epoch 2/12
469/469 [==============================] - 7s 15ms/step - loss: 2.2562 - accuracy: 0.1838 - val_loss: 2.2294 - val_accuracy: 0.3669
Epoch 3/12
469/469 [==============================] - 7s 16ms/step - loss: 2.2259 - accuracy: 0.2442 - val_loss: 2.1930 - val_accuracy: 0.5185
Epoch 4/12
469/469 [==============================] - 7s 15ms/step - loss: 2.1919 - accuracy: 0.2945 - val_loss: 2.1526 - val_accuracy: 0.5923
Epoch 5/12
469/469 [==============================] - 7s 15ms/step - loss: 2.1523 - accuracy: 0.3471 - val_loss: 2.1076 - val_accuracy: 0.6367
Epoch 6/12
469/469 [==============================] - 7s 15ms/step - loss: 2.1096 - accuracy: 0.3923 - val_loss: 2.0579 - val_accuracy: 0.6709
Epoch 7/12
469/469 [==============================] - 7s 15ms/step - 

# MLP

In [2]:
batch_size = 128
num_classes = 10
epochs = 20

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
shuffler = np.random.permutation(28*28)
for i, s in enumerate(x_train):
  x_train[i] = x_train[i].reshape(28*28,1)[shuffler].reshape((28, 28))
for i, s in enumerate(x_test):
  x_test[i] = x_test[i].reshape(28*28,1)[shuffler].reshape((28, 28))

In [10]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [12]:
# convert class vectors to binary class matrices

y_train = np_utils.to_categorical(y_train, num_classes) # each instance is an array 
y_test = np_utils.to_categorical(y_test, num_classes)

In [13]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# the warning message is expected

Epoch 1/20
469/469 [==============================] - 3s 6ms/step - loss: 0.2189 - accuracy: 0.9331 - val_loss: 0.1380 - val_accuracy: 0.9580
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0807 - accuracy: 0.9748 - val_loss: 0.0949 - val_accuracy: 0.9692
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0541 - accuracy: 0.9837 - val_loss: 0.0635 - val_accuracy: 0.9812
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0385 - accuracy: 0.9881 - val_loss: 0.0653 - val_accuracy: 0.9825
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0297 - accuracy: 0.9906 - val_loss: 0.0726 - val_accuracy: 0.9815
Epoch 6/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0237 - accuracy: 0.9925 - val_loss: 0.0996 - val_accuracy: 0.9780
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0189 - accuracy: 0.9942 - val_loss: 0.1022 - val_accuracy: 0.9789
Epoch 